In [1]:
cd ..

/Users/nicolas.peruchot/workdir/peafowl


In [2]:
from gensim.models import Word2Vec
from datasets import load_dataset
import pandas as pd
from peafowl.preprocessing.utils import lemmatizer_dataset
import umap
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors



In [3]:
data_tot = load_dataset('amazon_reviews_multi','en')['train']
data=data_tot['review_body']
stars=data_tot['stars']

data=pd.Series([e for i,e in enumerate(data) if i<10000 or i>189999])
stars=pd.Series([e for i,e in enumerate(stars) if i<10000 or i>189999])

Reusing dataset amazon_reviews_multi (/Users/nicolas.peruchot/.cache/huggingface/datasets/amazon_reviews_multi/en/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609)
100%|██████████| 3/3 [00:00<00:00, 285.06it/s]


In [4]:
data=lemmatizer_dataset(data)

In [5]:
model = Word2Vec(data, min_count=2,vector_size=300)

In [6]:
# reducer = umap.UMAP()
# embedding = reducer.fit_transform(model.wv.vectors)

In [29]:

word_to_vec=dict(zip(list(model.wv.index_to_key), model.wv.vectors))


In [22]:
model.wv.most_similar('great')

[('good', 0.9611092805862427),
 ('high', 0.9373072385787964),
 ('poor', 0.9363642334938049),
 ('low', 0.9316616654396057),
 ('decent', 0.928166925907135),
 ('terrible', 0.9269341826438904),
 ('overall', 0.924372673034668),
 ('headlamp', 0.9222519397735596),
 ('Vivitar', 0.9200717210769653),
 ('beat', 0.920066237449646)]

In [33]:
neigh = NearestNeighbors(n_neighbors=10)
neigh.fit(X=model.wv.vectors)

In [35]:
neigh.kneighbors(model.wv.get_vector("good").reshape(1,-1))

(array([[0.        , 0.8430347 , 0.89011824, 1.2856442 , 1.5460005 ,
         1.680331  , 1.6984603 , 1.7539138 , 1.7826484 , 1.8329241 ]],
       dtype=float32),
 array([[  6, 152, 162,   2,  50, 182,  80, 314, 177, 214]]))

In [38]:
model.wv.index_to_key[50]

'well'